In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../predictive_maintenance.csv")
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
UDI,10000.0,5000.50000,2886.895680,1.0,2500.75,5000.5,7500.25,10000.0
Air temperature [K],10000.0,300.00493,2.000259,295.3,298.30,300.1,301.50,304.5
Process temperature [K],10000.0,310.00556,1.483734,305.7,308.80,310.1,311.10,313.8
Rotational speed [rpm],10000.0,1538.77610,179.284096,1168.0,1423.00,1503.0,1612.00,2886.0
Torque [Nm],10000.0,39.98691,9.968934,3.8,33.20,40.1,46.80,76.6
Tool wear [min],10000.0,107.95100,63.654147,0.0,53.00,108.0,162.00,253.0
Target,10000.0,0.03390,0.180981,0.0,0.00,0.0,0.00,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


In [5]:
df.nunique()

UDI                        10000
Product ID                 10000
Type                           3
Air temperature [K]           93
Process temperature [K]       82
Rotational speed [rpm]       941
Torque [Nm]                  577
Tool wear [min]              246
Target                         2
Failure Type                   6
dtype: int64

In [6]:
df["Type"].value_counts()

Type
L    6000
M    2997
H    1003
Name: count, dtype: int64

In [7]:
df["Failure Type"].value_counts()

Failure Type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64

In [8]:
X = df.drop(['UDI','Product ID','Target',"Failure Type"],axis=1)
y = df['Failure Type']
X = pd.get_dummies(X).astype('int')
X

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_H,Type_L,Type_M
0,298,308,1551,42,0,0,0,1
1,298,308,1408,46,3,0,1,0
2,298,308,1498,49,5,0,1,0
3,298,308,1433,39,7,0,1,0
4,298,308,1408,40,9,0,1,0
...,...,...,...,...,...,...,...,...
9995,298,308,1604,29,14,0,0,1
9996,298,308,1632,31,17,1,0,0
9997,299,308,1645,33,22,0,0,1
9998,299,308,1408,48,25,1,0,0


In [9]:
X_columns = X.columns.to_numpy()

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[0.33333333, 0.375     , 0.22293364, ..., 0.        , 0.        ,
        1.        ],
       [0.33333333, 0.375     , 0.13969732, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.375     , 0.19208382, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.44444444, 0.375     , 0.27764843, ..., 0.        , 0.        ,
        1.        ],
       [0.44444444, 0.375     , 0.13969732, ..., 1.        , 0.        ,
        0.        ],
       [0.44444444, 0.375     , 0.19324796, ..., 0.        , 0.        ,
        1.        ]])

In [11]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y.max()

5

In [12]:
# Convert X and y to pandas DataFrame
X_df = pd.DataFrame(X_scaled, columns=X_columns)
y_df = pd.DataFrame(y, columns=['Target'])

# Concatenate X and y along columns
df = pd.concat([X_df, y_df], axis=1)

# Write DataFrame to CSV file
df.to_csv('processed_data.csv', index=False)